# Customer Segmentation for a Sports Facility

<div class="alert alert-block alert-info">

[1. Objectives](#1st-bullet)<br>
[2. Import Data/Libraries](#2nd-bullet)<br>  
[3. Data Exploration](#3rd-bullet)<br>  
[4. Data Visualization](#4th-bullet)<br> 
[5. Pre-Processing](#5th-bullet)<br> 


</div>

<div class="alert alert-block alert-success">

<a class="anchor" id="1st-bullet">    </a>
## 1. Objectives 
</div>

1. Explore the data and identify the variables that should be used to segment customers.
2. Identify customer segments
3. Justify the number of clusters chosen (taking in consideration the business use as well).
4. Explain the clusters found.
5. Suggest business applications for the findings and define general marketing approaches for each cluster.


<div class="alert alert-block alert-success">

<a class="anchor" id="2nd-bullet">    </a>
## 2. Import Libraries/Data
</div>

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import ceil #round number to closest integer

from ydata_profiling import ProfileReport

from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, RobustScaler
from sklearn.impute import KNNImputer

from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.stats import skew

import sompy
from sompy.visualization.mapview import View2D
from sompy.visualization.bmuhits import BmuHitsView
from sompy.visualization.hitmap import HitMapView

In [ ]:
data = pd.read_csv('XYZ_sports_dataset.csv', sep =';') 

<div class="alert alert-block alert-success">

<a class="anchor" id="3rd-bullet">    </a>
## 3. Data Exploration 
</div>

In [ ]:
data.head()

It is easily noted that our data contains missing values encoded as NaN.

#### Data Types:

In [ ]:
data.dtypes

- Date objects could be turned into a datetime type for easier manipulation and interpretation.
- Binary variables are all int/float types.

In [ ]:
'''
data['UseByTime'] = data['UseByTime'].astype("boolean")
data['AthleticsActivities'] = data['AthleticsActivities'].astype("boolean")
data['WaterActivities'] = data['WaterActivities'].astype("boolean")
data['FitnessActivities'] = data['FitnessActivities'].astype("boolean")
data['DanceActivities'] = data['DanceActivities'].astype("boolean")
data['TeamActivities'] = data['TeamActivities'].astype("boolean")
data['RacketActivities'] = data['RacketActivities'].astype("boolean")
data['CombatActivities'] = data['CombatActivities'].astype("boolean")
data['NatureActivities'] = data['NatureActivities'].astype("boolean")
data['SpecialActivities'] = data['SpecialActivities'].astype("boolean")
data['OtherActivities'] = data['OtherActivities'].astype("boolean")
data['HasReferences'] = data['HasReferences'].astype("boolean")
data['Dropout'] = data['Dropout'].astype("boolean")
'''

#### Missing Values:

In [ ]:
data.isna().sum()[data.isna().sum()!=0]

There are some missing values that need to be adressed during the detailed exploration of each variable.

#### Duplicated

In [ ]:
data.duplicated().sum()

There are no duplicated clients.

#### Descriptive Analysis:

In [ ]:
data.describe(include="all").T 

Looking at the descriptive analysis, some possible problems appear:
- For `Age` there seem to be some clients with age 0. The data is also skewed towards younger ages.
- For `Income`, some clients have value 0. It also seems like some extreme values appear for high values of income.
- The number of unique values shows that `LastPeriodStart` and `LastPeriodFinish` have some form of fixed dates.
- `DaysWithoutFrequency`,`LifetimeValue`, `NumberOfFrequencies`, `AttendedClasses`, `AllowedNumberOfVisits` and `RealNumberOfVisits` also seem to have extreme high values.

### Closer Look At Features:

#### ID:

In [ ]:
data['ID'].value_counts()

All ID's are unique so we can set it as the index:

In [ ]:
data.set_index('ID', inplace=True) 

In [ ]:
data.duplicated().sum()

We now have one duplicated entry that needs to be removed:

In [ ]:
data.drop_duplicates(inplace=True)

### Coherence Check

#### Age:

In [ ]:
np.sort(data['Age'].unique())

- All age values seem normal for a sport facility.
- Special attention to babies (0-3) and other childreen under 16 needs to be taken.

#### Income:

Children under 16 should have no income:

In [ ]:
data[data['Age']<16][data['Income']!=0]

There are 360 entries that need to have income set as 0:

In [ ]:
data.loc[data["Age"] < 16 , "Income"] = 0

#### Gender:

In [ ]:
data['Gender'].unique()

Values for `Gender` are normal.

#### EnrollmentStart / EnrollmentFinish / LastPeriodStart / LastPeriodFinish / DateLastVisit:

Dates should be DateTime format:

In [ ]:
data['EnrollmentStart'] = pd.to_datetime(data['EnrollmentStart'])
data['EnrollmentFinish'] = pd.to_datetime(data['EnrollmentFinish'])

data['LastPeriodStart'] = pd.to_datetime(data['LastPeriodStart'])
data['LastPeriodFinish'] = pd.to_datetime(data['LastPeriodFinish'])

data['DateLastVisit'] = pd.to_datetime(data['DateLastVisit'])

Some entries have enrollment start equal to enrollment finish:

In [ ]:
data[data['EnrollmentStart'] == data['EnrollmentFinish']]['Dropout'].value_counts()

All clients in this situation have 'Dropout' status set to 0.

In [ ]:
data[data['EnrollmentStart'] == data['EnrollmentFinish']]['LastPeriodFinish'].value_counts()

Most cases seem to be of clients with current active contracts (Finishing on '2019-12-31'). \
We need to take a deeper look at other cases:

#### Members who have not been active in current Period (from 2019-07-01 untill 2019-12-31):

Checking clients who have not been to the facility in the current period:

In [ ]:
data.loc[data['DateLastVisit']< '2019-06-30', 'Dropout'].value_counts()

In [ ]:
data[data['DateLastVisit']< '2019-06-30'].loc[data['Dropout']== 0][data['EnrollmentFinish'] != data['EnrollmentStart']]

The people who had DateLastVisit < '2019-06-30' and don't have a Dropout all have have 'EnrollmentStart'= 'EnrolmentFinish', which tells us that:
- these people have dropped out but have not been added to the system as dropouts.
- Their contract is still active as they've been paying, but not coming to the gym.

How can we deal with these inconsistencies?
1) If `DateLastVisit` matches the Last Period of activity (not current Period), clients are considered dropouts (all date are correct but Enrollment Dates) and have `EnrollmentFinish` on `DateLastVisit`.
2) Other clients are considered active: `LastPeriodFinish`should be '2019-12-31' and `EnrollmentFinish` on '2019-10-31'.


##### Dropouts:

We create a mask to select clients who follow condition **1** and should be considered dropouts:

In [ ]:
drop_mask = (
    (data['EnrollmentStart'] == data['EnrollmentFinish']) &
    (data['LastPeriodStart']<= data['DateLastVisit']) &
    (data['DateLastVisit']<= data['LastPeriodFinish']) &
    (data['DateLastVisit'] < pd.Timestamp(dt.date(2019,6,30)))
)
index_dropout = data.index[drop_mask].tolist()

`Dropout` should be set to 1 and `EnrollmentFinish` to the respective `DateLastVisit`

In [ ]:
data.loc[index_dropout, 'Dropout']=1

In [ ]:
data.loc[index_dropout, 'EnrollmentFinish'] = data.loc[index_dropout, 'DateLastVisit']

##### Non - Drop Out:

Looking at the non - dropout clients, instances where `LastPeriodFinish` is not '2019-12-31' will be considered errors and removed, as they contain too many discrepencies to be considered viable.

In [ ]:
mask = ( 
    (data['EnrollmentStart'] == data['EnrollmentFinish']) &
    (data['DateLastVisit']< '2019-06-30') 
)

In [ ]:
data[mask]['LastPeriodFinish'].value_counts()

In [ ]:
drop_mask = ( 
    (data['EnrollmentStart'] == data['EnrollmentFinish']) &
    (data['DateLastVisit']< '2019-06-30') &
    (data['LastPeriodFinish']!= pd.Timestamp(dt.date(2019,12,31)))
)
index_dropout = data.index[drop_mask].tolist()

In [ ]:
data.drop(index_dropout, inplace = True)

All other instances that have `EnrollmentStart` equal to `EnrollmentFinish` are considered active members:\
`LastPeriodFinish`should be '2019-12-31' and `EnrollmentFinish` on '2019-10-31'

In [ ]:
data.loc[data[data['EnrollmentFinish'] == data['EnrollmentStart']].index.tolist(), 'LastPeriodFinish']='2019-12-31'

In [ ]:
data.loc[data[data['EnrollmentFinish'] == data['EnrollmentStart']].index.tolist(), 'EnrollmentFinish']= '2019-10-31'

Clients who have a LastPeriod that doesn't match neither the:
- DateLastVisit: Last time client was at the facility
- EnrollmentFinish: End of contract

Are all dropouts that show inconsistencies in the date type variables:

In [ ]:
mask = (
    ~((data['LastPeriodStart'] <= data['EnrollmentFinish']) & (data['EnrollmentFinish'] <= data['LastPeriodFinish'])) &
    ~((data['LastPeriodStart'] <= data['DateLastVisit']) & (data['DateLastVisit'] <= data['LastPeriodFinish']))
)

In [ ]:
data.loc[data.index[mask].tolist()].Dropout.value_counts()

Depending on clustering results these clients can either be:
- Dropped totally from dataset as they represent 0.86% of entries;
- Left as they are;

In [ ]:
#data.drop(data.index[mask].tolist(), inplace = True)

### DaysWithoutFrequency

When reading the metadata, `DaysWithoutFrequency` is said to be a variable that has values only for dropouts.

Looking at our data, we see that this variable is calculated for all clients (even non dropouts), so we consider this variable with a new meaning:\
Days without frequency for all clients (until their current or last contract ended - `EnrollmentFinish`)

In [ ]:
data[data['Dropout'] == 0]['DaysWithoutFrequency'].value_counts()

Since we have changed values for `EnrollmentFinish`, these values will have to be recalculated to express correct values.

### Activities:

In [ ]:
activities = ['AthleticsActivities', 'WaterActivities','FitnessActivities','DanceActivities','TeamActivities','RacketActivities','CombatActivities','NatureActivities','SpecialActivities','OtherActivities']

We need to fix some inconsistencies:
- Until 3 years of age, clients can only do `WaterActivities`` or `OtherActivities``
- Until 16 years of age, clients can't do fitness activities.

Children **under three** only have `WaterActivites` and 3 entries with `FitnessActivities` (that are droped)

In [ ]:
data[data['Age']<3][activities].value_counts()

In [ ]:
mask = (
    (data['Age']<3)
    &(data['FitnessActivities'] ==1)
)

In [ ]:
data.drop(data.index[mask].tolist(), inplace = True)

Children **under 16** have 12 entries with `FitnessActivities` (we turn these values into 0)

In [ ]:
data[data['Age']<16]['FitnessActivities'].value_counts()

In [ ]:
mask = (
        (data['Age']<16)
    &(data['FitnessActivities'] !=0)
)

In [ ]:
data.loc[data.index[mask].tolist(), 'FitnessActivities'] = 0

### AllowedNumberOfVisitsBySLA

This variable represents the allowed number of visits, but is expressed in float form.\
 Turning it into the closest integer should be done so the variable is coherent with it's meta data.

In [ ]:
data['AllowedNumberOfVisitsBySLA']

### HasReference / NumberOfReferences

We need to make sure these two variables are coherent with each other:
- If clients has no refences, he can't have a number of references >0.
- If client has references, he can't have a number of references = 0.


In [ ]:
mask = (
    (data['HasReferences']!=1)
    &(data['NumberOfReferences']!=0)
)

In [ ]:
data[mask].shape

In [ ]:
data.loc[data.index[mask].tolist(), 'HasReferences'] = 1

Some clients have `HasReferences` set to 1 but `NumberOfReferences` equal to 0.\
We change `HasReferences` to 0 so these entries become coherent.

In [ ]:
mask = (
    (data['HasReferences']==1)
    &(data['NumberOfReferences']==0)
)

In [ ]:
data[mask].shape

In [ ]:
data.loc[data.index[mask].tolist(), 'HasReferences'] = 0

### LifetimeValue:

In [ ]:
data[data['LifetimeValue'] == 0]

**The variables will be further explored using visualizations:**

<div class="alert alert-block alert-success">

<a class="anchor" id="4th-bullet">    </a>
## 4. Data Visualization 
</div>

Visualizing our data can help us understand more about the distribution of the featues and find possible incoherences.

We start by separating metric, non-metric and date type features:

In [ ]:
metric_features = ['Age','Income','DaysWithoutFrequency','LifetimeValue','NumberOfFrequencies', 'AttendedClasses', 'AllowedWeeklyVisitsBySLA', 'AllowedNumberOfVisitsBySLA','RealNumberOfVisits','NumberOfRenewals','NumberOfReferences']
date_features = ['EnrollmentStart','EnrollmentFinish','LastPeriodStart','LastPeriodFinish','DateLastVisit']
non_metric_features = data.columns.drop(metric_features + date_features).to_list()

### Metric-Features

In [ ]:
# All Numeric Variables' Histograms in one figure
sns.set()

# Prepare figure. Create individual axes where each histogram will be placed
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(20, 11))

# Plot data
# Iterate across axes objects and associate each histogram (hint: use the ax.hist() instead of plt.hist()):
for ax, feat in zip(axes.flatten(), metric_features): # Notice the zip() function and flatten() method
    ax.hist(data[feat], bins = 4)
    ax.set_title(feat, y=-0.13)
    
# Layout
# Add a centered title to the figure:
title = "Numeric Variables' Histograms"

plt.suptitle(title)

""""
To save pictures at the end
if not os.path.exists(os.path.join('..', 'figures', 'exp_analysis')):
    # if the exp_analysis directory is not present then create it first
    os.makedirs(os.path.join('..', 'figures', 'exp_analysis'))
    
plt.savefig(os.path.join('..', 'figures', 'exp_analysis', 'numeric_variables_histograms.png'), dpi=200)
"""
plt.show()

In [ ]:
# All Numeric Variables' Box Plots in one figure
sns.set()

# Prepare figure. Create individual axes where each box plot will be placed
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(20, 11))

# Plot data
# Iterate across axes objects and associate each box plot (hint: use the ax argument):
for ax, feat in zip(axes.flatten(), metric_features): # Notice the zip() function and flatten() method
    sns.boxplot(x=data[feat], ax=ax)
    
# Layout
# Add a centered title to the figure:
title = "Numeric Variables' Box Plots"

plt.suptitle(title)
""""
# Save the figure
if not os.path.exists(os.path.join('..', 'figures', 'exp_analysis')):
    # if the exp_analysis directory is not present then create it first
    os.makedirs(os.path.join('..', 'figures', 'exp_analysis'))
    
plt.savefig(os.path.join('..', 'figures', 'exp_analysis', 'numeric_variables_boxplots.png'), dpi=200)
"""
plt.show()

By observing the above histograms and box plots, we notice that most variables have a skewed distribution that can symbolize the existence of outliers.
- `NumberOfReferences` and `AttendedClasses` are the most skewed ones.

This is extremely important information about our numeric variables, as these values can influence models negatively.\
Besides this behaviour, some other details were found:
- `DaysWithoutFrequency` has people with more than 4 years without frequency;
- `LifeTimeValue` seems to have clients with a total value of 0;

In [ ]:
# Pairwise Relationship of Numerical Variables
sns.set()

# Setting pairplot
sns.pairplot(data[metric_features], diag_kind="hist")

# Layout
plt.subplots_adjust(top=0.95)
plt.suptitle("Pairwise Relationship of Numerical Variables", fontsize=20)
""""
if not os.path.exists(os.path.join('..', 'figures', 'exp_analysis')):
    # if the exp_analysis directory is not present then create it first
    os.makedirs(os.path.join('..', 'figures', 'exp_analysis'))
    
plt.savefig(os.path.join('..', 'figures', 'exp_analysis', 'pairwise_relationship_of_numerical_variables.png'), dpi=200)
"""
plt.show()

We can see from the relationship plots above that:
- `Age` and `Income` seem to have a positive relation;
- `NumberOfFrequencies` and `AttendedClasses` seem to relate positively with `LifetimeValue`;

### Non-Metric-Features

In [ ]:
# All Non-Metric Variables' Absolute Frequencies
sns.set()

# Prepare figure. Create individual axes where each bar plot will be placed
fig, axes = plt.subplots(2, ceil(len(non_metric_features) / 2), figsize=(20, 11))

# Plot data
# Iterate across axes objects and associate each bar plot (hint: use the ax argument):
for ax, feat in zip(axes.flatten(), non_metric_features): # Notice the zip() function and flatten() method
    sns.countplot(x=data[feat].astype(object), ax=ax, color='#007acc')

title = "Categorical Variables' Absolute Frequencies"
plt.suptitle(title)

#plt.savefig(os.path.join('..', 'figures', 'exp_analysis', 'categorical_variables_frequecies.png'), dpi=200)
plt.show()

From observing the countplots, we notice that `DanceActivities` and `NatureActivities` have no entries set to 1.\
These variables can be considered uniformative and removed:

In [ ]:
data[['NatureActivities','DanceActivities']].value_counts()

In [ ]:
data.drop(['NatureActivities','DanceActivities'],axis=1, inplace=True)

In [ ]:
non_metric_features.remove('NatureActivities')
non_metric_features.remove('DanceActivities')

In [ ]:
activities.remove('NatureActivities')
activities.remove('DanceActivities')

We also look at the relation between age and certain activities:

In [ ]:
_=data.groupby("WaterActivities")["Age"].plot.hist(stacked=True,bins=60)
plt.title('Water Activities By Age')
plt.xlabel('Age')
plt.ylabel('Absolute Frequency')
plt.legend(title ='WaterActivities')
plt.show()

In [ ]:
sns.histplot(data=data, x='Age', hue='WaterActivities', bins = 35)

In [ ]:
sns.histplot(data=data, x='Age', hue='FitnessActivities', bins = 35)

<div class="alert alert-block alert-success">

<a class="anchor" id="5th-bullet">    </a>
## 5. Pre-Processing
</div>

## Outlier Removal

**Explain why comes first**

Our data is very skewed, so we're getting a very big number of outliers to remove using the standard methods and values, which is not acceptable consideting the problem in hand.\
To fight this problem, for the approaches used bellow, only very extreme outliers can be considered, which does not remove most detected outliers.

We combine different methods to remove outliers, as to get more robust results:

In [ ]:
data_original = data.copy()

#### Selecting outliers manually:

Looking at the distribution of our variables, we can select a threshold of values we consider extreme outliers:

In [ ]:
filters0 = (
    (data['NumberOfReferences']<2)
    &(data['NumberOfRenewals']<6)
    &(data['AttendedClasses']<400)
    &(data['LifetimeValue']<4000)
    &(data['RealNumberOfVisits']<60)
)

df_0 = data[filters0]
print('Percentage of data kept after removing outliers:', np.round(df_0.shape[0] / data_original.shape[0], 4))

##### IQR:

In [ ]:
q25 = data[metric_features].quantile(.25)
q75 = data[metric_features].quantile(.75)
iqr = (q75 - q25)

upper_lim = q75 + 1.5 * iqr
lower_lim = q25 - 1.5 * iqr

filters1 = []
for metric in metric_features:
    llim = lower_lim[metric]
    ulim = upper_lim[metric]
    filters1.append(data[metric].between(llim, ulim, inclusive='both'))

filters1 = pd.Series(np.all(filters1, 0))
filters1.index = data.index
df_1 = data[filters1]
print('Percentage of data kept after removing outliers:', np.round(df_1.shape[0] / data_original.shape[0], 4))

##### Z-Score:

In [ ]:
filters2 = []
for metric in metric_features:
    mean, std = np.mean(data[metric]), np.std(data[metric])
    z_score = np.abs((data[metric] - mean) / std)
    filters2.append(z_score < 3) 

filters2 = pd.Series(np.all(filters2, 0))
filters2.index = data.index
df_2 = data[filters2]
print('Percentage of data kept after removing outliers:', np.round(df_2.shape[0] / data_original.shape[0], 4))

In [ ]:
df_3 = data[(filters1 | filters2| filters0 )]
print('Percentage of data kept after removing outliers:', np.round(df_3.shape[0] / data_original.shape[0], 4))

Our robust approach gives us the same results as applying the manual filter.

Considering the skewness of our data, keeping this approach for an initial outlier removal is perfered, as we opt to remove a maximum of 1% of our data and both IQR and Z-Score tend to consider close to 5% of our data as very extreme outliers (making our threshold bigger converges to these values).

In [ ]:
data = df_0.copy()

## Missing Values (Data Imputation)

Only after our outilers are removed can we work of data imputation.

#### Metric Features- KNN Imputer:

For our numerical variables, we focus on using a KNNImputer to impute missing values:


In [ ]:
data_mv = data.copy()

In [ ]:
mv_metric_features = ['Income','NumberOfFrequencies','AllowedWeeklyVisitsBySLA']

Since we are working with a KNN model, we nedd to understand the optimal number of neighbours (K).\
 To do so, we apply an **Elbow Method** considering we want to optimize the skewness of our data:

In [ ]:
original_skewness = skew(data[mv_metric_features], axis=0, nan_policy='omit')

print(original_skewness)
k_values = range(1, 11)

skewness_difference = []

for k in k_values:
    imputer = KNNImputer(n_neighbors=k, weights="distance")
    imputed_data = imputer.fit_transform(data_mv[mv_metric_features])
    # so it ignores nans
    imputed_skewness = skew(imputed_data, axis=0, nan_policy='omit')
    
    skewness_diff = np.mean(np.abs(original_skewness - imputed_skewness))
    skewness_difference.append(skewness_diff)
print(skewness_difference)

plt.figure(figsize=(10, 6))
plt.plot(k_values, skewness_difference, marker='o')
plt.title('Skewness Difference vs K Value')
plt.xlabel('Number of Neighbors')
plt.ylabel('Average Skewness Difference')
plt.xticks(k_values)
plt.grid(True)
plt.show()



Best K seems to be 4, as it is the k for which we locate an "elbow".

In [ ]:
#rows with missing values:
nans_index = data[mv_metric_features].isna().any(axis=1)
data_mv[nans_index]

In [ ]:
imputer = KNNImputer(n_neighbors=5, weights='distance')
data_mv[mv_metric_features] = imputer.fit_transform(data_mv[mv_metric_features]).round(0)

For categorical features:

Looking at an example row, rows without any activities have sum = 0. \
If any of these rows have missing values in an activity, they will be filled with 1 so each client has at least 1 activity done in the sports facility:

In [ ]:
data_mv[activities].loc[24824]

In [ ]:
data_mv[activities].loc[24824].sum()

In [ ]:
for idx in data_mv.index[nans_index].tolist():
    if (data_mv.loc[idx, activities].sum() == 0) :
        data_mv.loc[idx, activities] = data_mv[activities].loc[idx].fillna(1)

In [ ]:
mv_categorical_features = ['AthleticsActivities','WaterActivities','FitnessActivities','TeamActivities','RacketActivities','CombatActivities','SpecialActivities','OtherActivities','HasReferences']

#### Non-Metric Features- Central Tendency:

Central tendency measures could also be applied to non-metric features, with the draw back that these methods possibly change distribution of features and can bring bias with it. Since our data is skewed, measured like median and mean can afect the distribution of our variables.

Since the number of missing values is small for each categorical feature (~0.3%), this approach is also correct and used:

In [ ]:
modes = data_mv[mv_categorical_features].mode().loc[0]
modes

In [ ]:
data_mv.fillna(modes, inplace=True) #replace non-metric features with mode
data_mv.isna().sum() 

#### We compare the skewness of the data before and after imputation:

In [ ]:
def calculate_skewness(data, variables, name):
    aux = []
    for feature in variables:
        aux.append(
            {
                'Feature': feature,
                'Skewness ' + name: data[feature].skew(),
            }
        )
    output = pd.DataFrame(aux)
    output.set_index('Feature', inplace = True)
    return output

Categorical features:

In [ ]:
data_skw = calculate_skewness(data, mv_categorical_features, "data")
data_mv_skw = calculate_skewness(data_mv, mv_categorical_features, "data_mv")
pd.concat([data_skw, data_mv_skw], join="outer", axis = 1)

Numerical features:

In [ ]:
data_skw = calculate_skewness(data, mv_metric_features, "data")
data_mv_skw = calculate_skewness(data_mv, mv_metric_features, "mv")
pd.concat([data_skw, data_mv_skw], join="outer", axis = 1)

#### Checking difference in distributions

**Diferenciar Gráficos**

In [ ]:
sns.set()

fig, axes = plt.subplots(2, 3, figsize=(20, 11))

for ax, feat in zip(axes[0].flatten(), mv_metric_features): 
    ax.hist(data[feat], bins = 40)
    ax.set_title(feat, y=-0.13)

for ax, feat in zip(axes[1].flatten(), mv_metric_features): 
    ax.hist(data_mv[feat], bins = 40)
    ax.set_title(feat, y=-0.13)
    
title = " Numeric Variables' Histograms"

plt.suptitle(title)
plt.show()

In [ ]:
sns.set()
fig, axes = plt.subplots(2, ceil(len(mv_categorical_features) / 2), figsize=(20, 11))

for ax, feat in zip(axes.flatten(), mv_categorical_features): 
    sns.countplot(x=data_mv[feat].astype(object), ax=ax, color='#007acc')

title = "Categorical Variables' Absolute Frequencies"
plt.suptitle(title)

plt.show()

In [ ]:
data = data_mv.copy()

----

<h2>Feature Engineering

### **Fixing Features**

1) Since we are talking about a number of visits, we transform `AllowedNumberOfVisitsBySLA` into an integer:

In [ ]:
data['AllowedNumberOfVisitsBySLA'] = np.round(data['AllowedNumberOfVisitsBySLA']).astype(int)

2) We decide to keep `DaysWithoutFrequency` as a variable thta applies to all clients.
Since we changed some values of `EnrollmentFinish`, we need to recalculate this feature so all values are correct:

In [ ]:
data['DaysWithoutFrequency'] = (data['EnrollmentFinish'] -data['DateLastVisit']).dt.days

3) We decide to turn our datetime features into the variables `Active_Period` (number of days of last period of activity) and `Contract_Duration`.

In [ ]:
data['Active_Period'] = (data['LastPeriodFinish']- data['LastPeriodStart'])
data['Active_Period']= data['Active_Period'].dt.days

In [ ]:
data['Contract_Duration'] = (data['EnrollmentFinish']- data['EnrollmentStart'])
data['Contract_Duration']=data['Contract_Duration'].dt.days

4) We need to check entries where `LifetimeValue` is 0.

Since we have only three entries, these are droped

In [ ]:
data.drop(data.index[data['LifetimeValue'] == 0].tolist(), inplace = True)

### **Feature Creation**

We try to create new features based on the features we were initially given.\
Our goal is to create as many new features as possible with the information we were given. These features are then selected based on their relevancy and redudancy.

1) Real number of visits in relation to the allowed number of visits

In [ ]:
data['PercentageOfVisits'] = ((data['RealNumberOfVisits'] / data['AllowedNumberOfVisitsBySLA'])).round(2)

2) Total number of Activities the client is signed in

In [ ]:
data['TotalNumberOfActivities'] = data.iloc[:, 11:19].sum(axis=1).astype(int)

Clients can't have no activity and `UseByTime` being equal to 0:

In [ ]:
data[data['TotalNumberOfActivities']==0]['UseByTime'].value_counts()

In [ ]:
data[data['TotalNumberOfActivities']==0]

In [ ]:
data.drop(data.index[data['TotalNumberOfActivities'] == 0].tolist(), inplace = True)

3) Monthly paid value

Since it is considered that a sport facility has monthly payments, it is important to understand how much a client pays each month:

In [ ]:
data['TotalMonths'] = (data['EnrollmentFinish'] - data['EnrollmentStart']) // np.timedelta64(1, 'M')
data['TotalMonths'] = np.where(data['TotalMonths'] <= 0, 1, data['TotalMonths']) #cases in which is less than one month, we will assume one month

data['MonthlyValue'] = (data['LifetimeValue'] / data['TotalMonths']).round(2)

data.drop('TotalMonths', axis=1, inplace=True) #drop total months column since we only needed it for this code

4) Percentage of visits that were classes

In [ ]:
data['PercentageOfClasses'] = (data['AttendedClasses'] / data['NumberOfFrequencies'] * 100).round(2)

5) Number of visits the client made to the facility during their contract

In [ ]:
data['Freq_Visits_Day']= (data['NumberOfFrequencies'] / data['Contract_Duration']).round(4)

6) Frequency of classes attended during contract duration

In [ ]:
data['Freq_Classes_Contract']= (data['AttendedClasses'] / data['Contract_Duration']).round(4)

7) Frequency of visits made during active period:

In [ ]:
data['Visits_ActivePeriod'] = (data['RealNumberOfVisits'] / data['Active_Period']).round(4)

We drop datetime features as they have no importance for clustering purposes:

In [ ]:
data.drop(['EnrollmentStart','EnrollmentFinish','LastPeriodStart','LastPeriodFinish','DateLastVisit'], axis =1, inplace = True)

### **Encoding**

From the analysis done previously, only the variable 'Gender' is not encoded. \
We chose to encode this variable into a bianry variable where 1= 'Female' and 0 = 'Male'

In [ ]:
ohe_data = pd.get_dummies(data, columns=['Gender'], dtype = int)

In [ ]:
ohe_data.drop('Gender_Male', axis = 1, inplace=True)

In [ ]:
data = ohe_data.copy()

### **Data Normalization/Scaling**

Since we have created new features, we re-define our lists of metric and non-metric features:

In [ ]:
non_metric_features = ['UseByTime','AthleticsActivities', 'WaterActivities', 'FitnessActivities','TeamActivities', 'RacketActivities', 'CombatActivities','SpecialActivities', 'OtherActivities','HasReferences','Dropout','Gender_Female']
metric_features = data.columns.drop(non_metric_features ).to_list()

### MinMaxScaler:

On a first approach, using a *MinMaxScaler* is considered:

In [ ]:
data_minmax = data.copy()

In [ ]:
scaler = MinMaxScaler()
scaled_feat = scaler.fit_transform(data_minmax[metric_features])
scaled_feat

In [ ]:
data_minmax[metric_features] = scaled_feat

### RobustScaler

Since we are dealing with skewed data with the presence of outliers outliers, using a `RobustScaler` is considered ideal as this scaler is not sensitive to outliers.

In [ ]:
data_robust = data.copy()

In [ ]:
scaler = RobustScaler()
scaled_feat = scaler.fit_transform(data_robust[metric_features])
scaled_feat

In [ ]:
data_robust[metric_features] = scaled_feat

We compare both scaling methods:

In [ ]:
sns.set()

fig, axes = plt.subplots(2, 3, figsize=(20, 11))

for ax, feat in zip(axes[0].flatten(), metric_features): 
    ax.hist(data_minmax[feat], bins = 40)
    ax.set_title(feat, y=-0.13)

for ax, feat in zip(axes[1].flatten(), metric_features): 
    ax.hist(data_robust[feat], bins = 40)
    ax.set_title(feat, y=-0.13)
    

title = "MinMax VS  Robust Numeric Variables' Histograms"

plt.suptitle(title)
plt.show()

Outliers cause the mean and standard deviation to soar to much higher values. The standard scaler uses these inflated values. Thus, it reduces the relative distance between outliers and other data points.

Hence when outliers are present, the standard scaler produces a distorted view of the original distribution.

Robust scaler doesn’t suffer from this defect. It resists the pull of outliers. Its scaled values have enough range so that the distance between outliers and other values remains largely intact.

In [ ]:
data = data_robust.copy()

## Variable Selection: Redundancy VS Relevancy

High correlation between variables will bias your results and give too much importance to these features.

#### Correlation Matrix:

In [ ]:
# Prepare figure
fig = plt.figure(figsize=(12, 12))

# Obtain correlation matrix. Round the values to 2 decimal cases. Use the DataFrame corr() and round() method.
corr = np.round(data[metric_features].corr(method="pearson"), decimals=2)

# Build annotation matrix (values above |0.5| will appear annotated in the plot)
mask_annot = np.absolute(corr.values)>= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape,"")) # Try to understand what this np.where() does

# Plot heatmap of the correlation matrix
sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

# Layout
fig.subplots_adjust(top=0.95)
fig.suptitle("Correlation Matrix", fontsize=20)

#plt.savefig(os.path.join('..', 'figures', 'exp_analysis', 'correlation_matrix.png'), dpi=200)

plt.show()

We uncover some important information:
- `Age` and  `Income` are highly correlated.
- `LifetimeValue` is also very correlated with `NumberOfRenewals`, `NumberOfFrequencies`, `AttendedClasses` and `ContractDuration`;
- `NumberOfFrequencies` is correlated with `NumberOfRenewals` and `ContractDuration`
- `AttendedClasses` is correlated with `PercentageOfClasses` and `Freq_Classes_Contract`
- `AllowedWeeklyVisitsBySLA` is highly negatively correlated with `PercentageOfClasses` and correlated with `AllowedNumberOfVisitsBySLA`
- `Visits_ActivePeriord` is highly correlated with `RealNumberOfVisits`.


1) We remove `Income` as we are looking to understand the different age groups in our sports facility.

In [ ]:
data.drop("Income", axis =1, inplace = True)

2) We remove `RealNumberOfVisits` as `Visits_ActivePeriod` is more relevant to study client behaviour as it is a value that is defined in the same time frame for all clients.

In [ ]:
data.drop("RealNumberOfVisits", axis =1, inplace = True)

3) We drop `DaysWithoutFrequency` as it has no correlation with other variables and can be considered "noise"

In [ ]:
data.drop("DaysWithoutFrequency", axis =1, inplace = True)

5) For the same reasons we drop `TotalNumberOfActivities`, `MonthlyValue`, `ActivePeriod` and `NumberOfReferences`

In [ ]:
data.drop(["TotalNumberOfActivities","MonthlyValue", "Active_Period", "NumberOfReferences" , "HasReferences"], axis =1, inplace = True)

Since we drop `NumberOfReferences` and `HasReferences` is related with this variable and highly unbalanced, we decide to drop it too.

6) We drop `LifetimeValue` as it has correlations with many other variable and can be considered redunctant.

In [ ]:
data.drop("LifetimeValue", axis =1, inplace = True)

7) Between `AttendedClasses` , `Freq_Classes_Contract` and `PercentageOfClasses`:

In [ ]:
data.drop("AttendedClasses", axis =1, inplace = True)

8) `AllowedWeeklyVisitsBySLA` and `AllowedNumberOfVisitsBySLA` and ``RealNumberOfVisits

In [ ]:
data.drop("AllowedWeeklyVisitsBySLA", axis =1, inplace = True)
data.drop("AllowedNumberOfVisitsBySLA", axis =1, inplace = True)

9) Between `NumberOfRenewals` and `ContractDuration`, the second variable is kept as we have no information about the renewal process of the company.

In [ ]:
data.drop("NumberOfRenewals", axis =1, inplace = True)

10) `NumberOfFrequencies` is droped once we have other variables such as `Freq_Visits_Day` that offers a more informative insight about the data

In [ ]:
data.drop("NumberOfFrequencies", axis =1, inplace = True)

when we have percentage of classes and freq, we keep both as we consider they offer differentet info.
Same thing for visits.

## Elbow Method

Do:
- General
- Activities acording to age (maybe others), gender
- Attendance


## Clustering by Perspectives

### Hierarchical Clustering

In [ ]:
data.columns

In [ ]:
metric_features = ['Age','Contract_Duration', 'PercentageOfVisits', 'PercentageOfClasses',
       'Freq_Visits_Day', 'Freq_Classes_Contract', 'Visits_ActivePeriod']

In [ ]:
# Detection of the reamining outliers trough DBSCAN
# epsilon of 2

from sklearn.neighbors import NearestNeighbors


neigh = NearestNeighbors(n_neighbors = (2 * len(metric_features)) - 1)
neigh.fit(data[metric_features])
distances, _ = neigh.kneighbors(data[metric_features])
distances = np.sort(distances[:, -1])
#plt.yticks(np.arange(1,22,1))
plt.plot(distances, color = 'green')
plt.show()

In [ ]:
from collections import Counter

from sklearn.cluster import DBSCAN


dbscan = DBSCAN(eps=1.8, min_samples = 2 * len(metric_features), n_jobs = -1)
dbscan_labels = dbscan.fit_predict(data[metric_features])

Counter(dbscan_labels)

In [ ]:
data = pd.concat([data, pd.Series(dbscan_labels, name = 'dbscan_labels', index = data.index)], axis =1)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame after preprocessing
# Replace 'df' with the name of your DataFrame variable

# Save the DataFrame to a CSV file
output_filename = "processed_dataset.csv"
data.to_csv(output_filename, index=False)  # Set 'index=False' to exclude row indices from the file

print(f"Dataset saved as {output_filename}")


In [ ]:
data_dbscan_out = data[dbscan_labels == -1]
data = data[dbscan_labels != -1]

In [ ]:

data.shape

## K means + hierarchial clustering

In [ ]:
data.columns

In [ ]:
attendance_perspective = [ 'Contract_Duration', 'PercentageOfVisits', 'PercentageOfClasses',
       'Freq_Visits_Day', 'Freq_Classes_Contract', 'Visits_ActivePeriod']
compare = ['UseByTime','Dropout', 'Gender_Female', 'Age']
df_activities = data[activities].copy()
df_attendance = data[attendance_perspective].copy()
df_compare = data[compare].copy()

In [ ]:
from sklearn import clone
from sklearn.cluster import KMeans


def get_ss(df):
    """Computes the sum of squares for all variables given a dataset
    """
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable

def r2(df, labels):
    sst = get_ss(df)
    ssw = np.sum(df.groupby(labels).apply(get_ss))
    return 1 - ssw/sst
    
def get_r2_scores(df, clusterer, min_k=2, max_k=10):
    """
    Loop over different values of k. To be used with sklearn clusterers.
    """
    r2_clust = {}
    for n in range(min_k, max_k):
        clust = clone(clusterer).set_params(n_clusters=n)
        labels = clust.fit_predict(df)
        r2_clust[n] = r2(df, labels)
    return r2_clust


# Set up the clusterers
kmeans = KMeans(
    init='k-means++',
    n_init=20,
    random_state=42
)

hierarchical = AgglomerativeClustering(
    metric='euclidean'
)

In [ ]:
# finding the optimal cluster on demo

In [ ]:
# Obtaining the R² scores for each cluster solution on demographic variables
r2_scores = {}
r2_scores['kmeans'] = get_r2_scores(df_attendance, kmeans)

for linkage in ['complete', 'average', 'single', 'ward']:
    r2_scores[linkage] = get_r2_scores(
        df_attendance, hierarchical.set_params(linkage=linkage)
    )

pd.DataFrame(r2_scores)

In [ ]:
############3

In [ ]:
kmeans_att = KMeans(n_clusters=35, init = 'k-means++', n_init=20, random_state=93)
km_att_labels = kmeans_att.fit_predict(df_attendance)

In [ ]:
df_km_att = pd.concat([df_attendance, pd.Series(km_att_labels, name='km_att_labels', index=df_attendance.index)], axis=1)

In [ ]:
centroids = pd.DataFrame(kmeans_att.cluster_centers_, columns = df_attendance.columns)


In [ ]:
hclust = AgglomerativeClustering(linkage='ward', affinity='euclidean', distance_threshold=0, n_clusters=None)
hclust_labels = hclust.fit_predict(centroids)

In [ ]:
df_km_att

In [ ]:
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)


for i, merge in enumerate(hclust.children_):
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            current_count += 1
        else:
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count


linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)


sns.set()
fig = plt.figure(figsize=(11,5))
y_threshold = 3
dendrogram(linkage_matrix, truncate_mode='level', labels=centroids.index, p=5, color_threshold=y_threshold, above_threshold_color='k')
plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - Dendrogram', fontsize=21)
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'Euclidean Distance', fontsize=13)
plt.show()

In [ ]:
# Re-running the Hierarchical clustering based on the correct number of clusters
hclust = AgglomerativeClustering(
    linkage='ward', 
    affinity='euclidean', 
    n_clusters=4
)
hclust_labels = hclust.fit_predict(centroids)
centroids['hclust_labels'] = hclust_labels

centroids  # centroid's cluster labels

In [ ]:
# Mapper between concatenated hierarchical clusters
cluster_mapper = centroids['hclust_labels'].to_dict()

# Mapping the hierarchical clusters on the centroids to the observations
df_km_att['kmeans_labels'] = df_km_att.apply(
    lambda row: cluster_mapper[(row['km_att_labels'])], axis=1)

df_km_att.drop('km_att_labels', axis=1, inplace=True)

In [ ]:
df_km_att.groupby('kmeans_labels').mean()

In [ ]:
def cluster_profiles(df, label_columns, figsize, compar_titles=None):
    """
    Pass df with labels columns of one or multiple clustering labels. 
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)
        
    sns.set()
    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2, figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)
        
        
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]
        
       
        pd.plotting.parallel_coordinates(centroids, label, color=sns.color_palette(), ax=ax[0])
        sns.barplot(x=label, y="counts", data=counts, ax=ax[1])

        
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(text=titl, xy=(0.95,1.1), xycoords='axes fraction', fontsize=13, fontweight = 'heavy') 
        ax[0].legend(handles, cluster_labels) 
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)), fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=-20)
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)), fontsize=13)
    
    plt.subplots_adjust(hspace=0.4, top=0.90)
    plt.suptitle("Cluster Simple Profilling", fontsize=23)
    plt.show()

In [ ]:
cluster_profiles(df_km_att, ['kmeans_labels'], (20,7))

In [ ]:
# cluster dois bue pouca frequencia, normal?

In [ ]:
df_activities.columns

In [ ]:
data["kmeans_labels"] = df_km_att['kmeans_labels']

In [ ]:
# A age nao pode estar obviamente aqui lol que burra

In [ ]:
df_compare_1 = data[['kmeans_labels',
              'UseByTime', 'Dropout', 'Gender_Female', 'Age']].groupby(['kmeans_labels']).sum()

df_compare_1

In [ ]:
df_compare_2 = data[['kmeans_labels',
              'AthleticsActivities', 'WaterActivities', 'FitnessActivities',
                'TeamActivities', 'RacketActivities', 'CombatActivities',
                'SpecialActivities', 'OtherActivities']].groupby(['kmeans_labels']).sum()

df_compare_2

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
df_compare_1.plot(kind='bar', stacked=False, ax=ax)
plt.xticks(rotation=0)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
df_compare_2.plot(kind='bar', stacked=False, ax=ax)
plt.xticks(rotation=0)
plt.show()

In [ ]:
#########3

In [ ]:
# Visualizing the R² scores for each cluster solution on demographic variables
pd.DataFrame(r2_scores).plot.line(figsize=(10,7))

plt.title("Attendance Variables:\nR² plot for various clustering methods\n", fontsize=21)
plt.legend(title="Cluster methods", title_fontsize=11)
plt.xlabel("Number of clusters", fontsize=13)
plt.ylabel("R² metric", fontsize=13)
plt.show()

In [ ]:
kmeans_att = KMeans(
    n_clusters=4,
    init='k-means++',
    n_init=20,
    random_state=42
)

In [ ]:
att_labels = kmeans_att.fit_predict(df_attendance)

In [ ]:
data["attendance_labels"] = att_labels

In [ ]:
(data["attendance_labels"]).value_counts()

In [ ]:
metric_features = ['Contract_Duration',
 'PercentageOfVisits',
 'PercentageOfClasses',
 'Freq_Visits_Day',
 'Freq_Classes_Contract',
 'Visits_ActivePeriod']

In [ ]:
df_centroids = data.groupby(['attendance_labels'])[metric_features].mean()

linkage = 'ward'
hclust = AgglomerativeClustering(
    linkage=linkage, 
    metric='euclidean', 
    distance_threshold=0, 
    n_clusters=None
)
hclust_labels = hclust.fit_predict(df_centroids)

In [ ]:
centroids = pd.DataFrame(kmeans_att.cluster_centers_, columns = df_attendance.columns)
hclust = AgglomerativeClustering(linkage='ward', affinity='euclidean', distance_threshold=0, n_clusters=None)

hclust_labels = hclust.fit_predict(centroids)

In [ ]:
counts = np.zeros(hclust.children_.shape[0])
n_samples = len(hclust.labels_)


for i, merge in enumerate(hclust.children_):
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            current_count += 1
        else:
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count


linkage_matrix = np.column_stack(
    [hclust.children_, hclust.distances_, counts]
).astype(float)


sns.set()
fig = plt.figure(figsize=(11,5))
y_threshold = 6
dendrogram(linkage_matrix, truncate_mode='level', labels=centroids.index, p=5, color_threshold=y_threshold, above_threshold_color='k')
plt.hlines(y_threshold, 0, 1000, colors="r", linestyles="dashed")
plt.title(f'Hierarchical Clustering - Dendrogram', fontsize=21)
plt.xlabel('Number of points in node (or index of point if no parenthesis)')
plt.ylabel(f'Euclidean Distance', fontsize=13)
plt.show()

In [ ]:
# Performing HC
hclust = AgglomerativeClustering(linkage='ward', metric='euclidean', n_clusters=5)
hc_labels = hclust.fit_predict(data[metric_features])
hc_labels

In [ ]:
# Characterizing the clusters
df_concat = pd.concat((data, pd.Series(hc_labels, name='labels', index=data.index)), axis=1)
df_concat.groupby('labels').mean()

------

### SOM + K-means clustering

In [ ]:
# This som implementation does not have a random seed parameter
# We're going to set it up ourselves
np.random.seed(42)

# Notice that the SOM did not converge - We're under a time constraint for this class
sm = sompy.SOMFactory().build(
    data[metric_features].values, 
    mapsize=[50, 50],  # NEEDS TO BE A LIST
    initialization='random',
    neighborhood='gaussian',
    training='batch',
    lattice='hexa',
    component_names=metric_features
)

## This will take a few minutes!
# sm.train(n_job=-1, verbose='info', train_rough_len=100, train_finetune_len=100)
sm.train(n_job=-1, verbose='info', train_rough_len=50, train_finetune_len=50)


In [ ]:
range_clusters = range(1, 11)

inertia = []
for n_clus in range_clusters:  # iterate over desired ncluster range
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=15, random_state=1)
    kmclust.fit(data[metric_features])
    inertia.append(kmclust.inertia_)  # save the inertia of the given cluster solution

In [ ]:
# The inertia plot
plt.figure(figsize=(9,5))
plt.plot(range_clusters, inertia)
plt.ylabel("Inertia: SSw")
plt.xlabel("Number of clusters")
plt.title("Inertia plot over clusters", size=15)
plt.show()

In [ ]:
# Perform K-Means clustering on top of the 2500 units (sm.get_node_vectors() output)
kmeans = KMeans(n_clusters=4, init='k-means++', n_init=20, random_state=42)
nodeclus_labels = kmeans.fit_predict(sm.codebook.matrix)
sm.cluster_labels = nodeclus_labels  # setting the cluster labels of sompy

hits = HitMapView(12, 12,"Clustering", text_size=10)
hits.show(sm, anotate=True, onlyzeros=False, labelsize=7, cmap="Pastel1")

plt.show()

### Profile Report: Might help us to get some info (delete later but refer if necessary)

In [ ]:
profile = ProfileReport(
    data, 
    title='Sports Facility Customer Data',
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
    },
)

In [ ]:
profile.to_notebook_iframe()